In [1]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB

In [5]:
import pandas as pd
from pycaret.classification import setup, compare_models, blend_models, stack_models, finalize_model, predict_model, save_model, create_model, tune_model

# Set a random seed for reproducibility
SEED = 42

# Load the data
train_file_path = 'train.csv'
test_file_path = 'test.csv'
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# Feature Engineering: Example of creating new features (you can add your custom logic here)
# train_data['New_Feature'] = train_data['Feature1'] * train_data['Feature2']

# Separate features and target variable in train data
data = train_data.drop(columns=['ID'])
data['Target'] = train_data['Target']

# Initialize PyCaret setup
clf_setup = setup(data=data, target='Target', session_id=SEED, use_gpu=True, verbose=False)

# Compare models and select the top 5 models
top5_models = compare_models(n_select=5)

# Tune the top 5 models
tuned_models = [tune_model(model) for model in top5_models]

# Blend the top models
blended = blend_models(estimator_list=tuned_models, fold=5)

# Create a more advanced meta model (e.g., a gradient boosting classifier)
meta_model = create_model('gbc')

# Stack the top models with the gradient boosting meta model
stacked_model = stack_models(estimator_list=tuned_models, meta_model=meta_model, fold=5)

# Finalize the stacked model (retrain on the entire dataset)
final_model = finalize_model(stacked_model)

# Preprocess the test data and make predictions
test_data_processed = predict_model(final_model, data=test_data.drop(columns=['ID']))

# Extract predictions from the processed data
test_data['Target'] = test_data_processed['prediction_label']

# Save the predictions to a CSV file
submission = test_data[['ID', 'Target']]
submission.to_csv('submission.csv', index=False)

# Save the final model for future use
save_model(final_model, 'final_model')

# Load the submission CSV file
submission_file_path = 'submission.csv'
submission = pd.read_csv(submission_file_path)

# Round the 'Target' values to the nearest integer
submission['Target'] = submission['Target'].round().astype(int)

# Save the reformatted submission file
submission.to_csv('reformatted_submission.csv', index=False, header=True)

print(submission)


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.7066,0.7597,0.7652,0.7320,0.7476,0.3977,0.3993,0.0700
lightgbm,Light Gradient Boosting Machine,0.6894,0.7725,0.7671,0.7098,0.7367,0.3593,0.3621,0.6490
gbc,Gradient Boosting Classifier,0.6781,0.7544,0.8608,0.6689,0.7523,0.3131,0.3348,2.2490
xgboost,Extreme Gradient Boosting,0.6749,0.7501,0.7435,0.7017,0.7218,0.3316,0.3327,0.4820
rf,Random Forest Classifier,0.6717,0.7171,0.7595,0.6928,0.7242,0.3207,0.3234,0.8160
dt,Decision Tree Classifier,0.6470,0.6424,0.6771,0.6942,0.6852,0.2835,0.2839,0.1220
et,Extra Trees Classifier,0.6453,0.6727,0.7349,0.6719,0.7014,0.2667,0.2691,0.6080
ada,Ada Boost Classifier,0.6319,0.6901,0.8010,0.6407,0.7116,0.2196,0.2310,0.6510
ridge,Ridge Classifier,0.5900,0.5787,0.8030,0.6047,0.6896,0.1194,0.1302,0.0750
lr,Logistic Regression,0.5809,0.5680,0.8011,0.5979,0.6845,0.0979,0.1070,0.7220


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6845,0.7648,0.6321,0.7701,0.6943,0.3748,0.3826
1,0.6667,0.7137,0.6571,0.7263,0.6900,0.3315,0.3334
2,0.7258,0.7979,0.6571,0.8214,0.7302,0.4584,0.4702
3,0.6882,0.7296,0.6667,0.7527,0.7071,0.3763,0.3795
4,0.6344,0.7463,0.6000,0.7079,0.6495,0.2729,0.2769
5,0.7258,0.7987,0.6415,0.8395,0.7273,0.4613,0.4783
6,0.6828,0.7611,0.5849,0.8052,0.6776,0.3805,0.3995
7,0.6667,0.7600,0.5849,0.7750,0.6667,0.3461,0.3599
8,0.7312,0.8165,0.6321,0.8590,0.7283,0.4742,0.4962


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6738,0.7373,0.8585,0.6642,0.7490,0.3043,0.3253
1,0.6559,0.7536,0.8667,0.6454,0.7398,0.2627,0.2887
2,0.6989,0.8046,0.8857,0.6788,0.7686,0.3587,0.3855
3,0.6559,0.7279,0.8476,0.6496,0.7355,0.2671,0.2871
4,0.6774,0.7286,0.8571,0.6667,0.7500,0.3149,0.3352
5,0.7204,0.8013,0.8396,0.7177,0.7739,0.4135,0.4223
6,0.6613,0.7218,0.8868,0.6483,0.7490,0.2651,0.2977
7,0.6720,0.7665,0.8868,0.6573,0.7550,0.2908,0.3221
8,0.6935,0.7726,0.8113,0.6992,0.7511,0.3582,0.3649


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6729,0.7449,0.7406,0.7009,0.7202,0.3274,0.3281
1,0.7043,0.7699,0.7062,0.7563,0.7304,0.4039,0.4050
2,0.6801,0.7596,0.7062,0.7233,0.7146,0.3508,0.3510
3,0.7016,0.7608,0.7630,0.7252,0.7436,0.3873,0.3880
4,0.6613,0.7593,0.7488,0.6840,0.7149,0.2998,0.3017
Mean,0.6840,0.7589,0.7329,0.7179,0.7248,0.3538,0.3547
Std,0.0166,0.0080,0.0230,0.0245,0.0110,0.0381,0.0379


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved
       ID  Target
0    2661       1
1    2662       1
2    2663       1
3    2664       1
4    2665       1
..    ...     ...
661  3322       0
662  3323       1
663  3324       1
664  3325       0
665  3326       1

[666 rows x 2 columns]
